In [1]:
# imports
#import Libraries
import numpy as np
from pydub import AudioSegment
import os
import glob
import tsfel
import csv
import pickle

In [3]:
#configuration
fs=44100        #audio sampling frequency
t=2             #total time of music sample

In [131]:
# Load the saved Orange model
with open('model_audio.pkcls', 'rb') as model_file:
    model = pickle.load(model_file)

In [133]:
# Print or use the model
print(model)

SklModelClassification(skl_model=SVC(coef0=1.0, gamma='auto', max_iter=100, probability=True))  # params={'C': 1.0, 'kernel': 'rbf', 'degree': 3, 'gamma': 'auto', 'coef0': 1.0, 'shrinking': True, 'probability': True, 'tol': 0.001, 'cache_size': 200, 'max_iter': 100}


In [141]:
#list all the mp3 in audio folder:
audio_folder = ".\\audio_files\\"
mp3_files = glob.glob(audio_folder+"*.mp3")

print('\n'.join('{} -> {}'.format(*k) for k in enumerate(mp3_files)))

#request user to selesct file
audio_n=input('Select the number of the audio file:')

0 -> .\audio_files\cozy-romantic-relaxing-podcast-jazz-music-212468.mp3
1 -> .\audio_files\easy-podcast-intro-theme-255102.mp3
2 -> .\audio_files\elevator-small-talk-255085.mp3


Select the number of the audio file: 0


In [145]:
#extract features
features=[]     #list of features

# Load MP3 file
print('Loading' + mp3_files[int(audio_n)])
audio = AudioSegment.from_file(mp3_files[int(audio_n)])

#convert to mono
audio = audio.set_channels(1)

# Convert to NumPy array
signal = np.array(audio.get_array_of_samples())

#extract part of the signal
#this is important speedup the process
n=3
s=signal[n*fs*t:(n+1)*(fs*t)]

#extract features
features.append(tsfel.feature_extraction.features.spectral_centroid(s, fs))
features.append(tsfel.feature_extraction.features.spectral_decrease(s, fs))
features.append(tsfel.feature_extraction.features.spectral_kurtosis(s, fs))
features.append(tsfel.feature_extraction.features.spectral_skewness(s, fs))
features.append(tsfel.feature_extraction.features.spectral_slope(s, fs))
features.append(tsfel.feature_extraction.features.spectral_spread(s, fs))
features.append(tsfel.feature_extraction.features.spectral_variation(s, fs))

print(features)


Loading.\audio_files\cozy-romantic-relaxing-podcast-jazz-music-212468.mp3
[1987.9531770558785, 0.0020221916410620416, 8.917202451102954, 2.5405630405294315, -5.057343567377034e-09, 3785.0560542992844, 0.16476461999674552]


In [163]:
#prediction
prediction = model(features)
print("Predicted class:", prediction)
print("Predicted class verbose:", mp3_files[prediction.astype(float).astype(int)])

Predicted class: 2.0
Predicted class verbose: .\audio_files\elevator-small-talk-255085.mp3
